In [2]:
from sklearn.decomposition import randomized_svd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
user = pd.read_csv('../Data/whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # shape(31176,5834) 유저 x 상품 아이템
user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # shape (31177, 3250) 유저 x 영상 아이템
null_del_user_place = pd.read_csv('../Data/whyout_data/null_del_user_place.csv')
null_del_user_product = pd.read_csv('../Data/whyout_data/null_del_user_product.csv')
null_del_user_video = pd.read_csv('../Data/whyout_data/null_del_user_video.csv')
index_null_del_user_place = pd.read_csv('../Data/whyout_data/index_null_del_user_place.csv', index_col=0)
index_null_del_user_product = pd.read_csv('../Data/whyout_data/index_null_del_user_product.csv', index_col=0)
index_null_del_user_video = pd.read_csv('../Data/whyout_data/index_null_del_user_video.csv', index_col=0)
normal_index_null_del_user_place = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_place.csv', index_col=0)
normal_index_null_del_user_product = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_product.csv', index_col=0)
normal_index_null_del_user_video = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_video.csv', index_col=0)

In [4]:
# 데이터프레임을 numpy matrix로 만듦
user_product_matrix = index_null_del_user_place.values

# 사용자의 평균 평점
user_ratings_mean = np.mean(user_product_matrix, axis=1)

# 사용자-상품에 대해 사용자 평균 평점을 뺀 것
matrix_user_mean = user_product_matrix - user_ratings_mean.reshape(-1,1)

In [5]:
pd.DataFrame(matrix_user_mean, columns = index_null_del_user_place.columns).head()

,1,2,3,4,5,6,7,8,9,10,...,4688,4689,4690,4691,4692,4693,4694,4695,4696,4697
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,0.999787,-0.000213,-0.000213,...,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213
2,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,1.993826,-0.006174,-0.006174,-0.006174,-0.006174,...,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,2.984671,-0.015329,-0.015329,0.984671,-0.015329,...,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329


In [6]:
# U 행렬, sigma 행렬, V 전치 행렬을 반환.
# U, sigma, Vt = svds(matrix, k = 20)
U, sigma, Vt = randomized_svd(user_product_matrix, n_components=20)

# 특이값 벡터를 대각 행렬 형태로 변환해줌
sigma = np.diag(sigma)

In [7]:
# U, Sigma, Vt의 행렬곱을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.matmul(np.matmul(U,sigma), Vt) + user_ratings_mean.reshape(-1,1)

In [8]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = index_null_del_user_place.columns)

In [14]:
def recommend_movies(df_svd_preds, user_id, df, ratings_df, num_recommendations):
    
    # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
    user_data = ratings_df.loc[user_id]  # Series 형태로 가져오기
    #print(user_data)

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = user_data[user_data > 0].index.tolist()
    print(user_history_indices)

    # SVD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    # 현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = df.loc[~df.index.isin(user_data[user_history_indices])]
    print(len(recommendations))
    print(len(df))

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    recommendations = recommendations.merge(predictions_df, left_index=True, right_on='index')

    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]

    return user_history_indices, recommendations

# 함수 사용 예시
already_rated, predictions = recommend_movies(df_svd_preds, 5, place, index_null_del_user_place, 10)

['6', '9', '326', '789', '949', '1355', '1520', '2322', '2370', '2417', '2713', '2716', '2717', '2719', '2726', '2760', '2786', '2789', '2794', '2800', '2801', '2809', '2811', '2823', '2824', '2842', '2857', '2866', '2882', '2904', '3006', '3007', '3116', '3206', '3207']
4694
4697


In [15]:
predictions

,idx,지역(위치정보),공간유형,주변환경,계절,테마,클릭 수,좋아요 수,북마크 수,댓글 수,index,Predictions
0,1521,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000235,0.000000,0.000000,0.0,1520,0.478856
1,2733,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0,0,0,1,0,0,0,0,0]","[0,1,0,1,0,0,0,0]",NaN,"[0,0,0,0,0,0,1,0,0,0,0,0,0,1,0]",0.052693,0.040541,0.036432,0.0,2732,0.355396
2,2709,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,0,1,1,0,0,0,1]",NaN,"[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]",0.289062,0.148649,0.135678,0.0,2708,0.345685
3,2633,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0,0,0,1,1,0,0,0,0]","[0,0,0,0,1,0,0,0]",NaN,"[0,0,0,0,0,0,0,0,0,0,0,1,0,1,0]",0.048748,0.045045,0.076633,0.0,2632,0.343418
4,728,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.001597,0.000000,0.002513,0.0,727,0.342852
5,3287,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,0,1,0,0,0,0,0]","[1,1,1,0]","[1,1,1,1,1,0,1,0,0,0,0,0,0,0,0]",0.058846,0.045045,0.055276,0.0,3286,0.341430
6,2996,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0,0,0,1,0,0,0,0,0]","[0,0,0,1,0,0,0,0]",NaN,"[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]",0.008172,0.004505,0.001256,0.0,2995,0.339398
7,3376,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0,0,0,0,0,1,0,0,0]","[0,0,0,0,0,1,0,0]",NaN,"[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]",0.006434,0.009009,0.002513,0.0,3374,0.330654
8,920,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000329,0.000000,0.000000,0.0,919,0.325908
9,2323,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,0.000845,0.000000,0.000000,0.0,2322,0.317577
